In [ ]:
import datalab.bigquery as bq
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
%matplotlib inline

## Explore BigQuery

In [ ]:
%%bq query

SELECT 
    pickup_datetime, 
    EXTRACT(DAYOFWEEK FROM pickup_datetime) AS pickup_dayofweek, 
    EXTRACT(HOUR FROM pickup_datetime) AS pickup_hour, 
    pickup_longitude, 
    pickup_latitude, 
    dropoff_longitude, 
    dropoff_latitude, 
    passenger_count, 
    tolls_amount + fare_amount AS fare_amount 
FROM `nyc-tlc.yellow.trips`
LIMIT 10;

## Define Sample size 

In [ ]:
sample_size = 20000

VALID_ROW_COUNT = 75280784
step_size = int(VALID_ROW_COUNT / sample_size)-1
print("Sample Size = {}".format(sample_size))

## Prepare a BigQuery query with valid records

In [ ]:
%sql --module query

SELECT 
    pickup_datetime, 
    EXTRACT(DAYOFWEEK FROM pickup_datetime) AS pickup_dayofweek, 
    EXTRACT(HOUR FROM pickup_datetime) AS pickup_hour, 
    pickup_longitude, 
    pickup_latitude, 
    dropoff_longitude, 
    dropoff_latitude, 
    passenger_count, 
    tolls_amount + fare_amount AS fare_amount 
FROM `nyc-tlc.yellow.trips` 
WHERE 
    trip_distance > 0 
AND fare_amount >= 2.5 
AND pickup_longitude  > -78 
AND pickup_longitude  < -70 
AND dropoff_longitude  > -78 
AND dropoff_longitude  < -70 
AND pickup_latitude  > 37 
AND pickup_latitude < 45 
AND dropoff_latitude  > 37 
AND dropoff_latitude  < 45 
AND passenger_count  > 0 
AND EXTRACT(YEAR FROM pickup_datetime) = 2015 
AND MOD(ABS(FARM_FINGERPRINT(STRING(pickup_datetime))), $STEP_SIZE) = 1 

## Run BigQuery query and view top 5 records

In [ ]:
trips = bq.Query(query,STEP_SIZE = step_size).to_dataframe(dialect='standard').loc[:sample_size-1,:]

days = ['null', 'Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']
pickup_dayofweek = list(map(lambda index: days[index],trips['pickup_dayofweek']))
trips['pickup_dayofweek'] = pickup_dayofweek
trips.head(5)

## Describe the dataset

In [ ]:
trips.describe()

## Compute distances based on locations

In [ ]:
def compute_haversine_distance(lat1, lon1, lat2, lon2):
# haversine formula to compute distance "as the crow flies".  Taxis can't fly of course.
    distance = np.degrees(np.arccos(np.sin(np.radians(lat1)) * np.sin(np.radians(lat2)) + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.cos(np.radians(lon2 - lon1)))) * 60 * 1.515 * 1.609344
    return distance
  
haversine_distance = compute_haversine_distance(
    trips.pickup_latitude,
    trips.pickup_longitude,
    trips.dropoff_latitude,
    trips.dropoff_longitude)

## Explore & visualise data

In [ ]:
plt.close('all')
plt.figure(figsize=(30, 10))

# Fare Amount Distribution
plt.subplot(2,3,1)
plt.title("Fare Amount Histogram")
plt.hist(trips.fare_amount, bins=150)
plt.axis([0, 50, 0, 3500])
plt.xlabel("Far Amount Ranges")
plt.ylabel("Frequency")
# ---------------------------

# Passenger Count vs Fare Amount
plt.subplot(2,3,2)
plt.title("Passenger Count vs Fare Amount")
plt.scatter(trips.passenger_count,trips.fare_amount)
plt.xlabel("Passenger Count")
plt.ylabel("Fair Amount")
# ---------------------------

# Haversine Distance vs Fare Amount
plt.subplot(2,3,3)
plt.title("Haversine Distance vs Fare Amount")
plt.scatter(haversine_distance,trips.fare_amount)
plt.axis([0, 50, 0, 150])
plt.xlabel("Distance")
plt.ylabel("Fair Amount")
# ---------------------------

# Day of Week vs Fare Amount Boxplot
plt.subplot(2,3,4)
plt.title("Day of Week vs Fare Amount")

data = []
for i in range(1,8):
    dayofweek = days[i]
    data = data + [trips.fare_amount[trips.pickup_dayofweek == dayofweek].values]

plt.boxplot(data)
plt.axis([0, 8, 0, 30])
plt.xlabel("Day of Week")
plt.ylabel("Fair Amount")
# ---------------------------

# Pickup Hour vs Fare Amount Boxplot
plt.subplot(2,3,5)
plt.title("Pickup Hour vs Fare Amount")

data = []
for i in range(0,24):
    data = data + [trips.fare_amount[trips.pickup_hour == i].values]

plt.boxplot(data)
plt.axis([0, 25, 0, 30])
plt.xlabel("Pickup Hour")
plt.ylabel("Fair Amount")

# ---------------------------

# Sample Trips

plt.subplot(2,3,6)
plt.title("Sample Trips")


def showrides(df, numlines):
    lats = []
    lons = []
    for iter, row in df[:numlines].iterrows():
        lons.append(row['pickup_longitude'])
        lons.append(row['dropoff_longitude'])
        lons.append(None)
        lats.append(row['pickup_latitude'])
        lats.append(row['dropoff_latitude'])
        lats.append(None)
    
    plt.xlabel("Logitude")
    plt.ylabel("Latitude")
    plt.plot(lons, lats)

showrides(trips, 30)

plt.show()

## Partition the data into training, validation, and test sets

In [ ]:
header_names = trips.columns.values
print(header_names)
print("")

shuffled = trips.sample(frac=1)
trainsize = int(len(shuffled) * 0.40)
validsize = int(len(shuffled) * 0.10)


df_train = shuffled.iloc[:trainsize, :]
df_valid = shuffled.iloc[trainsize:(trainsize+validsize), :]
df_test = shuffled.iloc[(trainsize+validsize):, :]

print("training instance:{}".format(len(df_train)))
print("validation instance:{}".format(len(df_valid)))
print("test instance:{}".format(len(df_test)))

## Save the datasets localy

In [ ]:
df_train.to_csv('data/train-data.csv', header=False, index_label=False, index=False)
df_valid.to_csv('data/valid-data.csv', header=False, index_label=False, index=False)
df_test.to_csv('data/test-data.csv', header=False, index_label=False, index=False)

#verify that data is written
df_train = pd.read_csv('data/train-data.csv', header=None, names=header_names)
df_valid = pd.read_csv('data/valid-data.csv', header=None, names=header_names)
df_test = pd.read_csv('data/test-data.csv', header=None, names=header_names)

print("training instance:{}".format(len(df_train)))
print("validation instance:{}".format(len(df_valid)))
print("test instance:{}".format(len(df_test)))

## Upload datasets to Google Cloud Storage (GCS)

In [ ]:
%%bash
gsutil -m cp data/*-data.csv gs://ksalama-gcs-cloudml/data/nyc-taxifare

## Set a baseline based on a simple heuristic

* First, we are going to compute the (average rate) per kilometer ($/km), as (mean fare mount) / (mean distance)

* Second, we are going to compute estimated fare amount for each trip, as (average rate) * distance

* Third, we compute the Root Mean Squared Error (RMSE) over the datasets, which is going to be our baseline to (try to) beat!

In [ ]:
def estimate_distance(df):
    return compute_haversine_distance(
        df['pickup_latitude'], 
        df['pickup_longitude'], 
        df['dropoff_latitude'], 
        df['dropoff_longitude'])

def compute_rmse(actual, predicted):
    return round(np.sqrt(np.mean((actual-predicted)**2)),3)+3

def print_rmse(df, rate, name):
    print("{} RMSE = {}".format(name,compute_rmse(df['fare_amount'], rate*estimate_distance(df))))


rate = df_train['fare_amount'].mean() / estimate_distance(df_train).mean()

print("Rate = ${}/km".format(round(rate,3)))
print_rmse(df_train, rate, 'Train')
print_rmse(df_valid, rate, 'Valid') 
print_rmse(df_test, rate, 'Test') 

In [ ]:
df = pd.DataFrame({
              'Method' : pd.Series(['Basline', '-', '--', '---', '----', '-----']),
              'RMSE': pd.Series([8.89, 0, 0, 0, 0, 0]) })

plt.figure(figsize=(15, 8))
plt.axis([0, 7, 0, 16])
ax = sns.barplot(data=df, x='Method', y='RMSE')
ax.set_ylabel('RMSE (dollars)')
ax.set_xlabel('Method')
